In [ ]:
#| hide
from sherlock.core import *
from fastcore.utils import first

# Sherlock Domains Python SDK

> AI agent & SDK to buy and managed domains from [Sherlock Domains](https://www.sherlockdomains.com/)

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/Fewsats/sherlock-python.git
```

```sh
$ pip install sherlock-domains
```


[repo]: https://github.com/Fewsats/sherlock-python
[pypi]: https://pypi.org/project/sherlock-domains/


## How to use

Create a Sherlock instance with a private key for the agent to use. If no key is provided, a new one will be generated and saved to the default config file.

In [ ]:
s = Sherlock()
s

Sherlock(pubkey=90ba884688884277e49080712f386eebc88806efa8345ca937f75fe80950156d)

You can search for a domain and request to purchase it. Purchasing a domain requires contact information as mandated by ICANN.

In [ ]:
sr = s.search("trakwiska.com")
sr

{'id': '5ef79dc9-20fa-469c-9fda-938330eb3056',
 'created_at': '2024-12-31T11:40:34.941Z',
 'available': [{'name': 'trakwiska.com',
   'tld': 'com',
   'tags': [],
   'price': 1105,
   'currency': 'USD',
   'available': True}],
 'unavailable': []}

In [ ]:
c = Contact(**{
    "first_name": "Test",
    "last_name": "User",
    "email": "test@example.com",
    "address": "123 Test St",
    "city": "Test City",
    "state": "CA",
    "country": "US",
    "postal_code": "12345",
})

s = Sherlock(c=c)
pr = s.request_purchase('trakwiska.com', sr['id'])

The request purchase will return different payment offers and payment methods. To buy this domain usign a credit card we do: 

In [ ]:
s.process_payment(pr['payment_request_url'], first(pr['offers'])['id'], 'credit_card', pr['payment_context_token'])

{'payment_method': {'checkout_url': 'https://checkout.stripe.com/c/pay/cs_live_a116wkFWokfHw73UFQI65GoWEAt4JTEja6OG3v5GOcb1zZljF9a0BABkm0#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl',
  'lightning_invoice': None},
 'expires_at': '2024-12-31T12:10:36.498Z'}

You can now use the checkout URL to complete the purchase and the domain will be registered to your agent.

You can view your domains with:

In [ ]:
s.domains()

[{'id': 'd9b2cc30-c15d-44b9-9d39-5d33da504484',
  'domain_name': 'h402.org',
  'created_at': '2024-12-28T18:58:49.899Z',
  'expires_at': '2024-12-31T18:58:42Z',
  'auto_renew': False,
  'locked': True,
  'private': True,
  'nameservers': [],
  'status': 'active'}]

Below is a list of all the tools that the client offers to manage the domains and purchases.

In [ ]:
s.as_tools().map(lambda t: t.__name__)

(#9) ['me','search','request_purchase','process_payment','domains','dns_records','create_dns','update_dns','delete_dns']

## AI agents

We will show how to enable your AI assistant to handle payments using [Claudette](https://claudette.answer.ai), Answer.ai convenient wrapper for Claude. You'll need to export your `ANTHROPIC_API_KEY`.

In [ ]:
from claudette import Chat, models


In [ ]:
import os
# os.environ['ANTHROPIC_LOG'] = 'debug'

In [ ]:
model = models[1]; model

'claude-3-5-sonnet-20240620'

Create a Sherlock instance with a public & private key for the agent to use.

Sherlock supports returning all the tools with `s.as_tools()`.

In order to request a purchase, we need to provide a contact information. You can either do so during the class initialization or manually. Then we just pass the extra tool.


In [ ]:
sp = 'You are a helpful assistant that has access to a domain purchase API.'
chat = Chat(model, sp=sp, tools=s.as_tools())

pr = f"Search if domain 'the-favourite-game.com' is available? If it is request a purchase and process the payment using credit card method."
r = chat.toolloop(pr, trace_func=print)
r

Message(id='msg_01A1MGtJxV4E4vdNUR79scBq', content=[TextBlock(text="Certainly! I'll search for the domain 'the-favourite-game.com', check its availability, and if it's available, I'll request a purchase and process the payment using a credit card. Let's go through this step by step.\n\nStep 1: Search for the domain\n\nFirst, let's search for the domain to check its availability and price.", type='text'), ToolUseBlock(id='toolu_01TqRV9hZ7nr3iaDCX72L556', input={'q': 'the-favourite-game.com'}, name='search', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 1195; Out: 135; Cache create: 0; Cache read: 0; Total: 1330)
Message(id='msg_01SHwRjnuVJW25WskvhRZzj9', content=[TextBlock(text="Great news! The domain 'the-favourite-game.com' is available. The price is 1105 cents USD, which is $11.05.\n\nStep 2: Request a purchase\n\nNow that we know the domain is available, let's request a purchase.", type=

Great! The payment process has been initiated successfully. Here's a summary of what has happened:

1. We searched for the domain 'the-favourite-game.com' and found it available for &#36;11.05.
2. We successfully requested a purchase for the domain.
3. We initiated the payment process using the credit card method.

To complete the purchase, you need to follow the checkout URL provided in the payment method response. Here's the URL:

https://checkout.stripe.com/c/pay/cs_live_a1KhVscSz9UKqqMKwCNAEbXGIwpMVYmz1ok35m6n7diGOEwnGM7f92Rkfb#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl

Please click on this link or copy and paste it into your browser to complete the payment process. You'll need to enter your credit card details and confirm the purchase.

The payment link will expire on December 31, 2024, at 12:10:47 UTC, so make sure to complete the payment before then.

Once you've completed the payment, the domain 'the-favourite-game.com' should be registered to your account. Is there anything else you'd like me to help you with regarding this domain purchase or any other domain-related questions?

<details>

- id: `msg_01DtYtGX2DgVfaircJN57BKS`
- content: `[{'text': "Great! The payment process has been initiated successfully. Here's a summary of what has happened:\n\n1. We searched for the domain 'the-favourite-game.com' and found it available for $11.05.\n2. We successfully requested a purchase for the domain.\n3. We initiated the payment process using the credit card method.\n\nTo complete the purchase, you need to follow the checkout URL provided in the payment method response. Here's the URL:\n\nhttps://checkout.stripe.com/c/pay/cs_live_a1KhVscSz9UKqqMKwCNAEbXGIwpMVYmz1ok35m6n7diGOEwnGM7f92Rkfb#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl\n\nPlease click on this link or copy and paste it into your browser to complete the payment process. You'll need to enter your credit card details and confirm the purchase.\n\nThe payment link will expire on December 31, 2024, at 12:10:47 UTC, so make sure to complete the payment before then.\n\nOnce you've completed the payment, the domain 'the-favourite-game.com' should be registered to your account. Is there anything else you'd like me to help you with regarding this domain purchase or any other domain-related questions?", 'type': 'text'}]`
- model: `claude-3-5-sonnet-20240620`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 2424, 'output_tokens': 524}`

</details>